In [ ]:
import torch
import pandas as pd
import os

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

import io

import ipywidgets as widgets
from PIL import Image
import io

import plotly.graph_objects as go
import plotly.express as px


import sys
sys.path.append("/mnt/raid/C1_ML_Analysis/source/us-famli-pl/src/")
from nets.classification import RopeEffnetV2s
from loaders.ultrasound_dataset import USAnnotatedBlindSweepDataModule

from tqdm.notebook import tqdm
import SimpleITK as sitk

def _frame_to_png_bytes(frame2d: np.ndarray) -> bytes:
    """Convert a (C,H,W) frame (any numeric dtype) to PNG bytes for ipywidgets.Image."""
    f = np.asarray(frame2d)
    # Handle NaNs/infs safely
    f = np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0)

    # Normalize to uint8
    fmin = float(f.min())
    fmax = float(f.max())
    if fmax > fmin:
        u8 = ((f - fmin) / (fmax - fmin) * 255.0).astype(np.uint8)
    else:
        u8 = np.zeros_like(f, dtype=np.uint8)

    img = Image.fromarray(u8, mode="RGB")  # RGB
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()


def visualize_sequence(
    images: np.ndarray,
    scores: np.ndarray,
    scores_pred: np.ndarray,
    *,
    fps: float = 10.0,
    title: str = "",
):
    """
    Jupyter-stable interactive viewer:
      - Left: grayscale frame shown via ipywidgets.Image
      - Right: Plotly line plots for GT + Pred with moving markers + vertical line
      - Controls: Play + Slider

    Parameters
    ----------
    images : np.ndarray
        Shape (T,H,W) or (C,T,H,W) or (T,H,W,C) supported if you tweak below.
        (Your code uses a helper _frame_to_png_bytes(frame) that expects a single frame.)
    scores : array-like
        Shape (T,), GT score per frame.
    scores_pred : array-like
        Shape (T,), predicted score per frame.
    """
    images = np.asarray(images)
    if images.ndim != 4:
        raise ValueError(f"`images` must have shape (T,H,W,C) (per your current code). Got {images.shape}")

    # Your original code says (C,T,H,W) but then unpacks as (T,H,W,C).
    # Keeping your current behavior: images.shape == (T,H,W,C)
    T, H, W, C = images.shape

    scores_np = np.asarray(scores).reshape(-1)
    if scores_np.shape[0] != T:
        raise ValueError(f"`scores` length must match T={T}. Got {scores_np.shape[0]}")

    scores_pred_np = np.asarray(scores_pred).reshape(-1)
    if scores_pred_np.shape[0] != T:
        raise ValueError(f"`scores_pred` length must match T={T}. Got {scores_pred_np.shape[0]}")

    # --- Widgets ---
    slider = widgets.IntSlider(value=0, min=0, max=T - 1, step=1, description="Frame", continuous_update=True)
    play = widgets.Play(
        value=0, min=0, max=T - 1, step=1,
        interval=int(1000 / max(fps, 1e-6)),
        description="Play",
    )
    widgets.jslink((play, "value"), (slider, "value"))

    # Image display (PNG bytes)
    img_w = widgets.Image(value=_frame_to_png_bytes(images[0]), format="png")

    # Metadata panel
    meta = widgets.HTML()
    def _meta_html(i: int) -> str:
        base = (
            f"<b>frame:</b> {i}"
            f"<br><b>score (gt):</b> {scores_np[i]:.3f}"
            f"<br><b>score (pred):</b> {scores_pred_np[i]:.3f}"
        )
        return base

    meta.value = _meta_html(0)

    # Plotly FigureWidget
    x = np.arange(T)
    fig = go.FigureWidget()

    # Lines
    fig.add_scatter(x=x, y=scores_np,      mode="lines", name="score (gt)")
    fig.add_scatter(x=x, y=scores_pred_np, mode="lines", name="score (pred)")

    # Current markers (one for each series)
    fig.add_scatter(x=[0], y=[float(scores_np[0])],      mode="markers", name="current (gt)")
    fig.add_scatter(x=[0], y=[float(scores_pred_np[0])], mode="markers", name="current (pred)")

    # Add max predicted score line
    max_pred_idx = int(np.argmax(scores_pred_np))
    fig.add_scatter(
        x=[max_pred_idx],
        y=[float(scores_pred_np[max_pred_idx])],
        mode="markers",
        name="max score (pred)",
        marker=dict(color="green", size=10, symbol="x"),
    )

    # y-range for the vertical line: cover both series
    y_min = float(np.min([scores_np.min(), scores_pred_np.min()]))
    y_max = float(np.max([scores_np.max(), scores_pred_np.max()]))

    fig.update_layout(
        title=title,
        xaxis_title="Frame",
        yaxis_title="Score",
        margin=dict(l=40, r=10, t=40, b=40),
        shapes=[
            dict(
                type="line",
                x0=0, x1=0,
                y0=y_min, y1=y_max,
                xref="x", yref="y",
                line=dict(width=2, dash="dash"),
            )
        ],
    )

    def _update(i: int):
        # Update image
        img_w.value = _frame_to_png_bytes(images[i])

        # Update markers + vertical line
        with fig.batch_update():
            # marker traces are indices 2 and 3
            fig.data[2].x = (i,)
            fig.data[2].y = (float(scores_np[i]),)

            fig.data[3].x = (i,)
            fig.data[3].y = (float(scores_pred_np[i]),)

            # keep line spanning both series ranges (or recompute if you want dynamic)
            fig.layout.shapes[0].update(x0=i, x1=i, y0=y_min, y1=y_max)

        # Update metadata
        meta.value = _meta_html(i)

    slider.observe(lambda ch: _update(ch["new"]), names="value")

    controls = widgets.HBox([play, slider])
    left = widgets.VBox([meta, img_w])
    right = widgets.VBox([fig])
    ui = widgets.VBox([controls, widgets.HBox([left, right])])

    return ui

In [ ]:
import numpy as np
import imageio.v3 as iio
import matplotlib.pyplot as plt

def _to_uint8_rgb_per_frame(frame: np.ndarray) -> np.ndarray:
    """
    Convert a single frame (H,W) or (H,W,C) to uint8 RGB using per-frame min/max,
    matching the spirit of your widget normalization.
    """
    f = np.asarray(frame)
    f = np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0)

    if f.ndim == 2:
        f = f[..., None]  # (H,W,1)
    if f.shape[-1] == 1:
        f = np.repeat(f, 3, axis=-1)
    elif f.shape[-1] != 3:
        raise ValueError(f"Expected 1 or 3 channels, got {f.shape[-1]}")

    fmin = float(f.min())
    fmax = float(f.max())
    if fmax > fmin:
        u8 = ((f - fmin) / (fmax - fmin) * 255.0).astype(np.uint8)
    else:
        u8 = np.zeros_like(f, dtype=np.uint8)

    return u8  # (H,W,3) uint8


def export_widget_like_mp4(
    images: np.ndarray,
    scores: np.ndarray,
    scores_pred: np.ndarray,
    out_mp4: str,
    *,
    fps: float = 10.0,
    title: str = "Sequence",
    show_filename: bool = False,
    filenames=None,
    dpi: int = 120,
):
    """
    Export a widget-like video:
      left: image + text metadata
      right: plot (gt + pred) + current markers + vertical line + max pred marker

    images: (T,H,W,C) or (T,H,W) or (T,H,W,1) or (T,H,W,3)
    scores, scores_pred: (T,)
    filenames: optional (T,)
    """
    imgs = np.asarray(images)
    if imgs.ndim == 3:  # (T,H,W)
        imgs = imgs[..., None]
    if imgs.ndim != 4:
        raise ValueError(f"images must be (T,H,W) or (T,H,W,C). Got {imgs.shape}")

    T, H, W, C = imgs.shape
    scores = np.asarray(scores).reshape(-1)
    scores_pred = np.asarray(scores_pred).reshape(-1)
    if scores.shape[0] != T or scores_pred.shape[0] != T:
        raise ValueError(f"scores and scores_pred must have length T={T}. Got {scores.shape[0]} and {scores_pred.shape[0]}")

    if show_filename:
        if filenames is None:
            raise ValueError("show_filename=True requires filenames=")
        filenames = np.asarray(filenames).reshape(-1)
        if filenames.shape[0] != T:
            raise ValueError(f"filenames must have length T={T}. Got {filenames.shape[0]}")

    x = np.arange(T)
    y_min = float(np.min([scores.min(), scores_pred.min()]))
    y_max = float(np.max([scores.max(), scores_pred.max()]))
    max_pred_idx = int(np.argmax(scores_pred))

    # Video writer settings
    # Note: imageio uses ffmpeg; install imageio-ffmpeg if needed.
    frames = []

    # Create one figure and update it per frame (faster than recreating).
    fig = plt.figure(figsize=(12, 5), dpi=dpi)
    gs = fig.add_gridspec(1, 2, width_ratios=[1.0, 1.6])

    ax_img = fig.add_subplot(gs[0, 0])
    ax_plot = fig.add_subplot(gs[0, 1])

    # --- Initialize left panel ---
    ax_img.axis("off")
    im_artist = ax_img.imshow(_to_uint8_rgb_per_frame(imgs[0]), interpolation="nearest")
    meta_text = ax_img.text(
        0.02, 0.98, "",
        transform=ax_img.transAxes,
        va="top", ha="left",
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.3", alpha=0.8)
    )

    # --- Initialize right panel plot ---
    ax_plot.set_title(title)
    ax_plot.set_xlabel("Frame")
    ax_plot.set_ylabel("Score")
    ax_plot.set_xlim(0, T - 1)
    ax_plot.set_ylim(y_min, y_max)

    (line_gt,) = ax_plot.plot(x, scores, label="score (gt)")
    (line_pred,) = ax_plot.plot(x, scores_pred, label="score (pred)")

    # current markers
    (cur_gt,) = ax_plot.plot([0], [float(scores[0])], marker="o", linestyle="None", label="current (gt)")
    (cur_pred,) = ax_plot.plot([0], [float(scores_pred[0])], marker="o", linestyle="None", label="current (pred)")

    # max pred marker
    ax_plot.plot([max_pred_idx], [float(scores_pred[max_pred_idx])], marker="x", linestyle="None", markersize=10, label="max score (pred)")

    # vertical line
    vline = ax_plot.axvline(0, linestyle="--")

    ax_plot.legend(loc="best")

    def _meta(i: int) -> str:
        s = f"frame: {i}\nscore (gt): {scores[i]:.3f}\nscore (pred): {scores_pred[i]:.3f}"
        if show_filename:
            s += f"\nfile: {filenames[i]}"
        return s

    meta_text.set_text(_meta(0))
    fig.tight_layout()

    # Render loop
    for i in range(T):
        im_artist.set_data(_to_uint8_rgb_per_frame(imgs[i]))
        meta_text.set_text(_meta(i))

        cur_gt.set_data([i], [float(scores[i])])
        cur_pred.set_data([i], [float(scores_pred[i])])
        vline.set_xdata([i, i])

        # draw -> RGB array
        fig.canvas.draw()
        w, h = fig.canvas.get_width_height()
        buf = np.asarray(fig.canvas.buffer_rgba())
        rgb = buf[..., :3].copy()  # drop alpha channel
        frames.append(rgb)

    plt.close(fig)

    frames_np = np.stack(frames, axis=0)  # (T,H,W,3)
    iio.imwrite(out_mp4, frames_np, fps=fps, codec="libx264", pixelformat="yuv420p")
    return out_mp4


In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis'

dev_id = 0
device = torch.device(f'cuda:{dev_id}' if torch.cuda.is_available() else 'cpu')

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v1.5/epoch=40-val_select=0.844.ckpt') # sigma 0.18, 0.12, 0.12, 0.1, 0.07)
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v1.5/epoch=48-val_select=0.805.ckpt')

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v1.6/epoch=39-val_select=0.770.ckpt')  # sigma -> 0.18, 0.12, 0.12, 0.1, 0.06
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v1.7/epoch=14-val_select=0.788.ckpt')
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v0.17/epoch=38-val_select=0.722.ckpt')




# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v0.15/epoch=27-val_select=0.699.ckpt')
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v0.16/epoch=39-val_select=0.576.ckpt')


# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v0.18/epoch=40-val_select=0.538.ckpt') # sigma -> Good 0.16 0.12 0.12 0.1 0.06

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v0.19/epoch=43-val_select=0.538.ckpt') # 0.14, 0.12, 0.12, 0.1, 0.08

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v1.8/epoch=44-val_select=0.769.ckpt') # Best one -> 0.14, 0.12, 0.12, 0.08, 0.04]

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v2.1/epoch=46-val_select=0.825.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.04

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v2.2/epoch=27-val_select=0.774.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.04 "reject_tau"  0.85

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v2.4/epoch=50-val_select=0.803.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.04 temporal_score_tv_weight 0.1
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v2.5/epoch=62-val_select=0.744.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.04 temporal_score_tv_weight 0.2

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v2.8/epoch=18-val_select=0.416.ckpt') 


# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v3.0/epoch=24-val_select=0.704.ckpt') 

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v3.4/epoch=19-val_select=0.825.ckpt')

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v4.1/epoch=36-val_select=0.856.ckpt')

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v5.0/epoch=16-val_select=0.802.ckpt') # 0.08, 0.12, 0.12, 0.09, 0.04

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v7.5/epoch=48-val_select=0.768.ckpt') # 0.09, 0.12, 0.12, 0.11, 0.06
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v7.6/epoch=32-val_select=0.760.ckpt') # 0.12 0.12 0.12 0.12 0.12

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v8.1/epoch=26-val_select=0.826.ckpt') # 0.09, 0.1, 0.1, 0.09, 0.08 top_aux [500, 2000] 0.1
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v8.2/epoch=41-val_select=0.738.ckpt') # 0.09, 0.12, 0.12, 0.1, 0.08 top_aux [500, 2000] 0.1

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v8.3/epoch=21-val_select=0.807.ckpt') # 0.14, 0.12, 0.12, 0.08, 0.04 top_aux 0, 2000 0.05
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v8.4/last.ckpt') # 0.14, 0.12, 0.12, 0.08, 0.04 top_aux [0, 2000] 0.1

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v8.6/epoch=22-val_select=0.705.ckpt') # 0.14, 0.12, 0.12, 0.08, 0.04 top_aux [0, 2000] 0.1

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.0/epoch=21-val_select=0.852.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.04 top_aux [0, 2000] 0.1

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.1/epoch=40-val_select=0.692.ckpt')  # 0.08 0.08 0.08 0.08 0.06
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.2/epoch=19-val_select=0.817.ckpt') # 0.08, 0.16, 0.12, 0.08, 0.04 top_aux [0, 2000] 0.1
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.3/epoch=20-val_select=0.743.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.06 top_aux 0.05 [2000, 4000

# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.8/epoch=16-val_rec_top3=0.363.ckpt') # 0.16, 0.12, 0.12, 0.08, 0.04 
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v9.9/epoch=33-val_rec_top3=0.405.ckpt') # 0.1, 0.16, 0.12, 0.08, 0.04




# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v10.0/epoch=38-val_select=0.398.ckpt') # 0.1, 0.12, 0.12, 0.08, 0.04 top_aux 0 -> Very Good!
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v10.1/epoch=58-val_select=0.396.ckpt') # 0.14, 0.12, 0.1, 0.08, 0.06 top_aux 0
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/v10.3/epoch=52-val_rec_top3=0.371.ckpt') # 0.08, 0.12, 0.12, 0.08, 0.04 top_aux 0
# model_fn = os.path.join(mount_point, 'train_output/classification/RopeEffnet/', 'v10.4/epoch=60-val_select=0.383.ckpt') # 0.1, 0.12, 0.1, 0.08, 0.04]

model = RopeEffnetV2s.load_from_checkpoint(model_fn, map_location=device).eval() 

In [ ]:
model.hparams

In [ ]:
dm = USAnnotatedBlindSweepDataModule(**model.hparams)
dm.setup()

In [ ]:
def run_test(dm, model):
    
    output_csv = model_fn.replace('.ckpt', '_predictions.csv')

    if os.path.exists(output_csv):
        print(f"Predictions file {output_csv} already exists. Loading it.")
        df = pd.read_csv(output_csv)
        return df
    
    test_dl = dm.test_dataloader()
    test_ds = dm.test_ds

    df_scores = []
    with torch.no_grad():
        for idx, X_d in tqdm(enumerate(test_dl), total=len(test_dl)):
            file_path = test_ds.df.iloc[idx]['file_path']
            logits = model(X_d['img'].permute(0, 2, 1, 3, 4).to(device))  # [B, N, C]
            probs = torch.softmax(logits, dim=-1)
            levels = torch.tensor([0.0, 0.25, 0.5, 0.75, 1.0], device=probs.device)
            score = (probs * levels).sum(dim=-1)  # [B, N]

            df = pd.DataFrame({
                'frame_index': list(np.arange(X_d['img'].shape[2])),
                'file_path': file_path,
                'score_pred': score.squeeze().cpu().numpy(),
                'score': X_d['scalar'].squeeze().cpu().numpy(),
            })
            df_scores.append(df)    
    df = pd.concat(df_scores, ignore_index=True)

    if not os.path.basename(model_fn) == "last.ckpt":
        df.to_csv(output_csv, index=False)
        print(f"Saved predictions to {output_csv}")
    return df

df_test_pred = run_test(dm, model)

In [ ]:
df_test_pred['error'] = df_test_pred['score_pred'] - df_test_pred['score']
df_test_pred['abs_error'] = df_test_pred['error'].abs()

In [ ]:
df_test_pred['abs_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
bins = np.array([0.0, 0.25, 0.5, 0.75, 1.0])

def to_class(x):
    return np.argmin(np.abs(bins - x))

df_test_pred["y_true"] = df_test_pred.score.apply(to_class)
df_test_pred["y_pred"] = df_test_pred.score_pred.apply(to_class)

In [ ]:
import itertools

cm = confusion_matrix(df_test_pred['y_true'], df_test_pred['y_pred'])
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8,6))
plt.imshow(cm_norm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(bins))

fmt = '.3f' 
thresh = cm_norm.max() / 2.
for i, j in itertools.product(range(cm_norm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm_norm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm_norm[i, j] > thresh else "black")

plt.xticks(tick_marks, [f"{b:.2f}" for b in bins])
plt.yticks(tick_marks, [f"{b:.2f}" for b in bins])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()





In [ ]:
report = classification_report(df_test_pred['y_true'], df_test_pred['y_pred'], target_names=['0.0', '0.25', '0.5', '0.75', '1.0'])
print(report)

In [ ]:
m = df_test_pred.score == 1.0
plt.hist(df_test_pred.score_pred[m], bins=30)
plt.axvline(1.0, color="r")
plt.title("Predictions for true high_measurable (1.0)")
plt.show()

In [ ]:
for t in [0.7, 0.8, 0.9]:
    tp = np.mean(df_test_pred.score_pred[df_test_pred.score == 1.0] >= t)
    print(f"P(score_pred >= {t} | true=1.0) = {tp:.3f}")


In [ ]:
for t in [0.5, 0.9]:
    tp = np.mean(df_test_pred.score_pred[df_test_pred.score == 0.0] >= t)
    print(f"P(score_pred >= {t} | true=0.0) = {tp:.3f}")

In [ ]:
df_seq = (
    df_test_pred.groupby("file_path")
      .agg(score_max=("score","max"), pred_max=("score_pred","max"))
      .reset_index()
)


In [ ]:
px.violin(df_seq, y="pred_max", x="score_max", box=True, points="all", title=f"Predicted max score vs GT max score per sequence - sigma {model.hparams.sigma}")

In [ ]:
px.violin(df_test_pred, y="score_pred", x="score", box=True, points="outliers", title=f"Predicted score vs GT score per frame - sigma {model.hparams.sigma}")

In [ ]:
for s in [0, 0.25, 0.5, 0.75, 1.0]:
    print(f"Score: {s}")
    print(df_test_pred.query(f'score == {s}')['score_pred'].describe(percentiles=[.01, .1, .25, 0.5, 0.75, 0.9, 0.95, 0.99]))

In [ ]:
test_ds = dm.test_ds
df_frames = test_ds.df_frames
df = test_ds.df

In [ ]:
list(df_seq.query('score_max >= 0.75 and pred_max <= 0.25')['file_path'].drop_duplicates())

In [ ]:
file_paths = df_test_pred.query('score_pred >= 0.45 and score == 0.0')['file_path'].drop_duplicates()
print(len(file_paths))
idx = 36
file_path = file_paths.iloc[idx]

print(file_path)
q = df_test_pred.query('file_path == @file_path')
score, score_pred = q['score'], q['score_pred']

img = sitk.ReadImage(os.path.join(mount_point, file_path))
img_np = sitk.GetArrayFromImage(img)  # [T,H,W]

if img.GetNumberOfComponentsPerPixel() == 1:
    img_np = np.expand_dims(img_np, -1).repeat(3, axis=-1)

visualize_sequence(img_np, score, score_pred, title=f"{'/'.join(file_path.split('/')[-2:])}")

In [ ]:
out_fn = os.path.join(os.path.dirname(model_fn), os.path.splitext(os.path.basename(file_path))[0]) + ".mp4"

export_widget_like_mp4(
    images=img_np,                # (T,H,W,C) or (T,H,W)
    scores=score,
    scores_pred=score_pred,
    out_mp4=out_fn,
    fps=10.0,
    title=file_path,
)

In [ ]:

def run_example():

    file_path = df_frames.query('annotation_label == "high_measurable" or annotation_label == "low_measurable" or annotation_label == "high_visible" and tag != "AC"')['file_path'].drop_duplicates().sample(n=1).values[0]
    
    print(file_path)
    idx = df.query(f'file_path == "{file_path}"').index[0] 
    X_d = test_ds[idx]

    with torch.no_grad():
        logits = model(X_d['img'].unsqueeze(0).permute(0, 2, 1, 3, 4).to(device))
        probs = torch.softmax(logits, dim=-1)
        levels = torch.tensor([0.0, 0.25, 0.5, 0.75, 1.0], device=probs.device)
        score = (probs * levels).sum(dim=-1)  # [B, N]


    visualize_sequence(X_d['img'].permute(1,2,3,0), X_d['scalar'], score.squeeze(0).cpu().numpy(), title=f"{'/'.join(file_path.split('/')[-2:])}")

In [ ]:
w_bins = torch.arange(1, 6, dtype=torch.float32) 
w_bins = w_bins / w_bins.mean()

In [ ]:
w_bins

In [ ]:
c = np.array([369458, 92519, 15579, 7674, 5970])
w = 1.0 / c
w = w / w.mean()
w


In [ ]:
wb = np.array([0.2,0.4,0.6,1.0,2.0])
wb = wb / wb.mean()
wb

In [ ]:
y = torch.tensor([0.0, 0.0, 0.25, 1.0, 0.75, 0.0, 0.0])
s = torch.tensor([0.1, 0.2, 0.3, 0.9, 0.6, 0.2, 0.1])

ds = s[1:] - s[:-1]
dy = y[1:] - y[:-1]
print(ds)
print(dy)
d = ds - dy 
print(d)
print(np.abs(d).mean())

In [ ]:
np.abs(d).mean()